In [1]:
from datasets import load_dataset
from io import BytesIO
import torch
import torchaudio
from spauq.core.metrics import spauq_eval

In [2]:
dataset = load_dataset("danjacobellis/aria_ea_audio_preprocessed",split='validation').with_format("torch")

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

In [5]:
def mp3(sample):
    fs=48000
    audio = sample['audio'].permute(1,0)
    config = torchaudio.io.CodecConfig(qscale=9)
    size_bytes = []
    recovered = []
    for channel in audio:
        with BytesIO() as f:
            torchaudio.save(f, channel.unsqueeze(0), format="mp3", sample_rate=fs, compression=config)
            size_bytes.append(len(f.getvalue()))
            f.seek(0)
            recovered.append(torchaudio.load(f,format="mp3")[0])
    recovered = torch.cat(recovered)
    eval_output = spauq_eval(reference=audio,estimate=recovered,fs=fs)
    sample['SSR'] = eval_output['SSR']
    sample['SRR'] = eval_output['SRR']
    sample['compression_ratio'] = 2*audio.numel()/sum(size_bytes)
    return sample

In [9]:
test = dataset.select(range(0,1200,100)).map(mp3)

In [13]:
test['compression_ratio'].mean()

tensor(17.9869)

In [14]:
test['SRR'].mean()

tensor(7.0687)

In [15]:
test['SSR'].mean()

tensor(18.1476)